# Hausaufgabe
translate(Hausaufgabe) -> **Homework**

Halo!
На семинаре мы создали простую seq2seq модель на основе rnn для перевода, а сейчас постараемся засунуть туда attention. Работать будем с тем же датасетом DE->EN (датасеты получше просто не влезают в память колаба, но если у вас есть CPU+тонна времени или GPU побольше, то можно попробовать построить перевод на WMT14 или IWSLT )

В конце домашней работы предполагается написание отчета о проделанной работе.

In [0]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
import numpy as np

import spacy

import random
import math
import time

from torchtext.datasets import TranslationDataset, Multi30k #WMT14, IWSLT
from torchtext.data import Field, BucketIterator

import torch.nn.functional as F

In [0]:
seed = 43

random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [54]:
! python -m spacy download en
! python -m spacy download de


spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [0]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

# немецкий язык является полем SRC, а английский в поле TRG
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [0]:
# В датасете содержится ~ 30к предложений средняя длина которых 11
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'),  fields = (SRC, TRG))

Давайте посмотрим что у нас с датасетом и сделаем словари для SRC и TGT

In [57]:
labels = ['train', 'validation', 'test']
dataloaders = [train_data, valid_data, test_data]
for d, l in zip(dataloaders, labels):
    print("Number of sentences in {} : {}".format(l, len(d.examples)))


Number of sentences in train : 29000
Number of sentences in validation : 1014
Number of sentences in test : 1000


In [58]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)
print("Number of words in source vocabulary", len(SRC.vocab))
print("Number of words in source vocabulary", len(TRG.vocab))

Number of words in source vocabulary 7855
Number of words in source vocabulary 5893


## Encoder

Энкодер будет ровно как в семинаре, с кдинственным изменением -- forward будет возвращать не только hidden, cell, но еще и outputs. Это нужно (надеюсь, вы уже поняли) для использования attention в декодере

In [0]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence #попробовать эти методы

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout, bidirectional = False):
        """
        :param: input_dim is the size/dimensionality of the one-hot vectors that will be input to the encoder. This is equal to the input (source) vocabulary size.
        :param: emb_dim is the dimensionality of the embedding layer. This layer converts the one-hot vectors into dense vectors with emb_dim dimensions.
        :param: hid_dim is the dimensionality of the hidden and cell states.
        :param: n_layers is the number of layers in the RNN.
        :param: percentage of the dropout to use
        
        """
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim)
 
        self.rnn = nn.LSTM(input_size=emb_dim, num_layers = n_layers, hidden_size=hid_dim, bidirectional = bidirectional) #попробовать поменять на lstm или gru
                                                                                           #не забывать про bidirectional
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        """
        :param: src sentences (src_len x batch_size)
        """
        # embedded = <TODO> (src_len x batch_size x embd_dim)
        embedded = self.embedding(src)
        # dropout over embedding
        embedded = self.dropout(embedded)
        outputs, (hidden, cell) = self.rnn(embedded)
        # [Attention return is for lstm, but you can also use gru]
        return outputs, hidden, cell

In [22]:
a = torch.tensor([[1,2,3], [1,2,0], [1,0,0]])
pack_padded_sequence(a, [3,2,1])

PackedSequence(data=tensor([1, 2, 3, 1, 2, 1]), batch_sizes=tensor([3, 2, 1]), sorted_indices=None, unsorted_indices=None)

## Decoder
Оп ля, а тут уже что-то новенькое

Мы будем реализовывать attention, который будет смотреть из tgt в src (НЕ self-attention). 

Определим два класса -- Attention и DecoderAttn. Мы разбили их на два класса, чтобы можно было играться с типом внимания, не меняя код DecoderAttn. Как вы помните с лекции, в качестве аттеншена можно брать любую странную функцию (конкатенация, маленькая сеточка, ...), и все будет работать! Поэтому вам предлагается попробовать несколько разных.


---------------------
Есть два подхода к реализации аттеншена:

Подход #1:

1. Вычисляется embed
2. На основе hidden c прошлого шага, embedded и (возможно) enc_out вычисляется attention, а точнее, веса attention (поэтому не забудьте softmax!!). Размерность batch_size * max_len, max_len -- максимальная длина предложения в батче, т.е. shape[0] от выхода энкодера.
3. К enc_out применяется attention: чаще всего dot product от enc_out и attention_weights (не забудьте про измерение батч. Чтобы нормально вычислить dot_product по батчу, вам поможет torch.bmm)
4. Берутся attention и embedded и сворачиваются в один вектор размерности такой, чтобы кормить его self.lstm. Например, это можно сделать с помощью обычного линейного слоя
5. Вычисляется новое скрытое состояние new_hidden. Это наша self.lstm, примененная к выходу пункта 4.
6. Вычисляется prediction, как в семинаре

Грубо говоря, вся разница с семинаром в том, что мы вместо того, чтобы embedded пихать в self.lstm, миксуем аттэншен на основе всего, что имеем (enc_out, hidden, embedded) и запихиваем в self.lstm микс аттэншена и embedded.

![alt text](https://i.imgur.com/cmkRY0r.png)


Подход #2:

1. Вычисляется embed
2. Вычисляется output, new_hidden (строчка output, (hidden, cell) = self.rnn(embedded, (hidden, cell)))
3. На основе output и enc_out вычисляется attention, а точнее, веса attention (поэтому не забудьте softmax!!)
3. К enc_out применяется attention: чаще всего dot product от enc_out и attention_weights (не забудьте про измерение батч. Чтобы нормально вычислить dot_product по батчу, вам поможет torch.bmm)
4. Вычисляется prediction на основе attention и output. Можно, например, взять nn.Linear() от конкатенации attention и output.

Разница с первым подходом в том, что мы сначала вычисляем выход rnn слоя, а потом смотрим вниманием на src и на основе выхода rnn и attn считаем выход (prediction). 

![alt text](https://i.imgur.com/5aWjQWv.png)


Вам предлагается реализовать хотя бы 1 из вариантов и хотя бы 2 варианта функции attention (в классе Attention)


**Attention метод 1**

In [0]:
class Attention(nn.Module):
    def __init__(self,n_layers,hid_dim,device, method="one-layer-net", bidirectional = False): # add parameters needed for your type of attention
        super(Attention, self).__init__()
        self.method = method # attention method you'll use. e.g. "cat", "one-layer-net", "dot", ...
        self.linear = nn.Linear(2*hid_dim*(bidirectional+1), 1)
        self.linear_hidden = nn.Linear(n_layers*hid_dim*(bidirectional+1), hid_dim*(bidirectional+1))
        self.bidirectional = bidirectional
        self.hid_dim = hid_dim
        self.device = device
        self.n_layers = n_layers #1 или 2 слоя
        
    def forward(self, last_hidden, embedding, seq_len=None):
        if self.n_layers > 1:
          last_hidden = last_hidden[-self.n_layers*(1+self.bidirectional):]
          curr = last_hidden[0]
          for i in range(1, self.n_layers*(1+self.bidirectional)):
            hidden = torch.cat((curr,last_hidden[i]), dim = -1)
            curr = hidden
          last_hidden = torch.tanh(self.linear_hidden(curr))

        else:
          last_hidden = last_hidden[-(1+self.bidirectional):]
          if self.bidirectional:
            last_hidden = torch.cat((last_hidden[0],last_hidden[1]), dim = -1)

        if self.method == 'one-layer-net':
          """input = torch.zeros((encoder_outputs.shape[0], encoder_outputs.shape[1], 1)).to(self.device)#переделать, размер батча не всегда одинаковый
          for ix in range(len(encoder_outputs)):
            input[ix] = self.linear(torch.cat((encoder_outputs[ix], last_hidden), dim = -1))"""
          input = self.linear(torch.cat((embedding, last_hidden), dim = -1)) #(seq_len, batch, input_size), (1, batch, hid_dim*(bidirectional+1))
          return F.softmax(torch.tanh(input), dim = 0) #надо еще раз посмотреть потому что веса(1,1,1,1)

        elif self.method == 'dot':
          input = torch.sum(last_hidden.unsqueeze(0)*embedding, dim = -1).unsqueeze_(-1)
          return F.softmax(input, dim = 0)

        else:
          raise ValueError(self.method, 'method does not exist')
        
class DecoderAttn(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, attention, bidirectional = False, dropout=0.1):
        super(DecoderAttn, self).__init__()
        
        self.emb_dim = hid_dim*(bidirectional+1)
        self.hid_dim = hid_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        self.bidirectional = bidirectional
        
        self.attn = attention # instance of Attention class

        # define layers
        self.embedding = nn.Embedding(output_dim,hid_dim*(bidirectional+1))
        
        self.rnn = nn.LSTM(self.emb_dim, hid_dim, n_layers, bidirectional = bidirectional) #(lstm embd, hid, layers, dropout), попробовать gru
        self.out = nn.Linear(hid_dim*(bidirectional+1), self.output_dim) # Projection :hid_dim x output_dim
        self.linear_embed = nn.Linear(2*self.hid_dim*(1+bidirectional), hid_dim*(bidirectional+1))
        self.dropout = nn.Dropout(dropout)
        #self.embed_linear = nn.Linear(self.hid_dim*(1+bidirectional) + self.emb_dim, self.hid_dim)

        # more layers you'll need for attention
        
    def forward(self, input, hidden, cell, encoder_output):
        # make decoder with attention
        # use code from seminar notebook as base and add attention to it
        input_ = input.unsqueeze(0)
        
        # (1 x batch_size x emb_dim)
        embedded = self.embedding(input_)# embd over input and dropout 
        embedded = self.dropout(embedded)

        attn_weights = self.attn(hidden, embedded)
        new_embed = torch.sum(encoder_output*attn_weights, dim = 0) # (num_layers * num_directions, batch, hidden_size)
        #assert hidden.shape == new_hidden.shape
        #new_hidden = self.embed_linear(torch.cat((new_hidden, embedded), dim = -1))#для n_layers > 1 наверное придется продублировать 1 ось
        #output, (hidden, cell) = self.rnn(embedded, (torch.cat((new_hidden, new_hidden), dim =0) if self.bidirectional else new_hidden, cell)) #сюда надо подавать ((1+bidirectional), batch_size, ...)
        new_embed = self.linear_embed(torch.cat((new_embed.unsqueeze(0), embedded), dim=-1))
        output, (hidden, cell) = self.rnn(new_embed, (hidden, cell))
        prediction = self.out(output.squeeze(0))
        return prediction, hidden, cell, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

**Attention метод 2**

In [0]:
class Attention(nn.Module):
    def __init__(self,n_layers,hid_dim,device, method="one-layer-net", bidirectional = False): # add parameters needed for your type of attention
        super(Attention, self).__init__()
        self.method = method # attention method you'll use. e.g. "cat", "one-layer-net", "dot", ...
        self.linear = nn.Linear(2*hid_dim*(bidirectional+1), 1)#попробовать вместо 1 hid_dim
        self.linear_hidden = nn.Linear(n_layers*hid_dim*(bidirectional+1), hid_dim*(bidirectional+1))
        self.bidirectional = bidirectional
        self.hid_dim = hid_dim
        self.device = device
        self.n_layers = n_layers #1 или 2 слоя
        
    def forward(self, last_hidden, encoder_outputs, seq_len=None):
        if self.n_layers > 1:
          last_hidden = last_hidden[-self.n_layers*(1+self.bidirectional):]
          curr = last_hidden[0]
          for i in range(1, self.n_layers*(1+self.bidirectional)):
            hidden = torch.cat((curr,last_hidden[i]), dim = -1)
            curr = hidden
          last_hidden = torch.tanh(self.linear_hidden(curr)) #попробовать tanh()

        else:
          last_hidden = last_hidden[-(1+self.bidirectional):]
          if self.bidirectional:
            last_hidden = torch.cat((last_hidden[0],last_hidden[1]), dim = -1)

        if self.method == 'one-layer-net':
          """input = torch.zeros((encoder_outputs.shape[0], encoder_outputs.shape[1], 1)).to(self.device)#переделать, размер батча не всегда одинаковый
          for ix in range(len(encoder_outputs)):
            input[ix] = self.linear(torch.cat((encoder_outputs[ix], last_hidden), dim = -1))"""
          input = self.linear(torch.cat((encoder_outputs, last_hidden.repeat(encoder_outputs.shape[0],1).reshape(encoder_outputs.shape)), dim = -1))
          return F.softmax(torch.tanh(input), dim = 0) #надо еще раз посмотреть потому что веса(1,1,1,1)

        elif self.method == 'dot':
          input = torch.sum(last_hidden.unsqueeze(0)*encoder_outputs, dim = -1).unsqueeze_(-1)
          return F.softmax(input, dim = 0)

        else:
          raise ValueError(self.method, 'method does not exist')
        
class DecoderAttn(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, attention, bidirectional = False, dropout=0.1):
        super(DecoderAttn, self).__init__()
        
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        self.bidirectional = bidirectional
        
        self.attn = attention # instance of Attention class

        # define layers
        self.embedding = nn.Embedding(output_dim,emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, bidirectional = bidirectional) #(lstm embd, hid, layers, dropout), попробовать gru
        self.out = nn.Linear(2*self.hid_dim*(1+bidirectional), self.output_dim) # Projection :hid_dim x output_dim
        self.dropout = nn.Dropout(dropout)
        #self.embed_linear = nn.Linear(self.hid_dim*(1+bidirectional) + self.emb_dim, self.hid_dim)

        # more layers you'll need for attention
        
    def forward(self, input, hidden, cell, encoder_output):
        # make decoder with attention
        # use code from seminar notebook as base and add attention to it
        input_ = input.unsqueeze(0)
        
        # (1 x batch_size x emb_dim)
        embedded = self.embedding(input_)# embd over input and dropout 
        embedded = self.dropout(embedded)

        attn_weights = self.attn(hidden, encoder_output)
        new_hidden = torch.sum(encoder_output*attn_weights, dim = 0) # (num_layers * num_directions, batch, hidden_size)
        #assert hidden.shape == new_hidden.shape
        #new_hidden = self.embed_linear(torch.cat((new_hidden, embedded), dim = -1))#для n_layers > 1 наверное придется продублировать 1 ось
        #output, (hidden, cell) = self.rnn(embedded, (torch.cat((new_hidden, new_hidden), dim =0) if self.bidirectional else new_hidden, cell)) #сюда надо подавать ((1+bidirectional), batch_size, ...)
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        new_hidden = new_hidden.repeat(output.shape[0],1)
        new_hidden = new_hidden.reshape(output.shape)
        output = torch.cat((new_hidden, output), dim=-1)
        prediction = self.out(output.squeeze(0))
        return prediction, hidden, cell, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [147]:
a = torch.randn(2, 32, 2*30)
b = torch.randn(2, 32, 2*30)
torch.tanh(a).requires_grad

False

## Seq2Seq module

Здесь опять ничего не поменяется кроме того, что энкодер теперь возвращает свой output, а декодер его принимает

In [0]:
BOS_IDX = SRC.vocab.stoi['<sos>']

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        # Hidden dimensions of encoder and decoder must be equal
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self._init_weights() 
    
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        """
        :param: src (src_len x batch_size)
        :param: tgt
        :param: teacher_forcing_ration : if 0.5 then every second token is the ground truth input
        """
        
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        out,hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0, :]
        
        for t in range(1, max_len):
            
            output, hidden, cell, _ = self.decoder(input, hidden, cell, out) #TODO pass state and input throw decoder 
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            input = (trg[t] if teacher_force else top1)
        
        return outputs
    
    def translate(self, src, input):   

        max_len = input.shape[0]
        batch_size = input.shape[1]
     
        outputs = torch.zeros(max_len, batch_size, self.decoder.output_dim).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        src = torch.tensor(src).to(self.device)
        out, hidden, cell =  self.encoder(src)# TODO pass src throw encoder
        
        #first input to the decoder is the <sos> tokens
        input = input[0, :]
        weights = []
        
        model.eval()
        for t in range(1, max_len):
            with torch.no_grad():
              output, hidden, cell, weights_ = self.decoder(input, hidden, cell, out) #TODO pass state and input throw decoder 
            outputs[t] = output
            top1 = output.max(1)[1]
            input = (top1)
            weights.append(weights_)
        
        return outputs, weights
    
    def _init_weights(self):
        p = 0.08
        for name, param in self.named_parameters():
            nn.init.uniform_(param.data, -p, p)

In [0]:
input_dim = len(SRC.vocab)
output_dim = len(TRG.vocab)
src_embd_dim =  tgt_embd_dim = 256
hidden_dim = 512
num_layers =  2
dropout_prob = 0.5

batch_size = 64
PAD_IDX = TRG.vocab.stoi['<pad>']

iterators = BucketIterator.splits((train_data, valid_data, test_data),
                                  batch_size = batch_size, device = device)
train_iterator, valid_iterator, test_iterator = iterators

enc = Encoder(input_dim, src_embd_dim, hidden_dim, num_layers, dropout_prob, True)
attn = Attention(num_layers, hidden_dim,device, bidirectional=True)#попробовать поиграться с attention
dec = DecoderAttn(output_dim, tgt_embd_dim, hidden_dim, num_layers, attn,True, dropout_prob)
model = Seq2Seq(enc, dec, device).to(device)

In [151]:
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2, bidirectional=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): DecoderAttn(
    (attn): Attention(
      (linear): Linear(in_features=2048, out_features=1, bias=True)
      (linear_hidden): Linear(in_features=2048, out_features=1024, bias=True)
    )
    (embedding): Embedding(5893, 1024)
    (rnn): LSTM(1024, 512, num_layers=2, bidirectional=True)
    (out): Linear(in_features=512, out_features=5893, bias=True)
    (linear_embed): Linear(in_features=2048, out_features=1024, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [0]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output= model(src, trg)
        
        
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing !!
            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)


            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [0]:
#import telegram_send
#!telegram-send --configure

In [174]:
from IPython.display import clear_output
import matplotlib.pyplot as plt

max_epochs = 30
CLIP = 1

# TODO
optimizer = optim.Adam(model.parameters(), weight_decay = 1e-4)
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)
scheduler = optim.lr_scheduler.StepLR(optimizer, 3, gamma=0.5)

best_valid_loss = float('inf')
history_val = []
history_train = []

for epoch in range(max_epochs):
    
    
    train_loss = round(train(model, train_iterator, optimizer, criterion, CLIP), 5)
    valid_loss = round(evaluate(model, valid_iterator, criterion),5)
    history_val.append(valid_loss)
    history_train.append(train_loss)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model_one_layer_4layer_bid.pt')
    
    scheduler.step()

    clear_output(True)
    plt.plot(history_train, label = 'train_loss')
    plt.plot(history_val, label = 'valid_loss')
    """plt.savefig('image.jpg')
    !telegram-send --image image.jpg """
    plt.show()

    print('Epoch: {} \n Train Loss {}  Val loss {}:'.format(epoch, train_loss, valid_loss))
    print('Train Perplexity {}  Val Perplexity {}:'.format(np.exp(train_loss), np.exp(valid_loss)))
#!telegram-send 'done'

RuntimeError: ignored

In [166]:
model.load_state_dict(torch.load('model_one_layer_4layer_bid.pt'))

<All keys matched successfully>

In [167]:
test_loss = evaluate(model, test_iterator, criterion)

print('| Test Loss: {} Test PPL:{}|'.format(test_loss, np.exp(test_loss)))

| Test Loss: 3.328218385577202 Test PPL:27.88861066463126|


In [0]:
EOS_IDX = SRC.vocab.stoi['<eos>']

def translate(sentence, max_size = 10):
    """
    function that uses .translate() method of the model to translate german sentence into english
    params:
        sentence: tokenized gernam sentence
    """
    sent_vec = SRC.process([sentence]).to(device)
    input = torch.zeros((max_size, 1)).type(torch.LongTensor).to(device)
    input += SRC.vocab.stoi['<sos>']
    translation_idx, weights = model.translate(sent_vec, input)
    vyvod = []
    for t in translation_idx:
        if t[0].max(0)[1] != EOS_IDX:
            vyvod.append(TRG.vocab.itos[t[0].max(0)[1]])
        else:
            break
    return vyvod, weights

In [168]:
translate(["<sos>","ein", "klein", "apfel", "<eos>"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


(['<unk>', 'a', 'is', 'is', 'a', '.', '.'],
 [tensor([[[1.]]], device='cuda:0'),
  tensor([[[1.]]], device='cuda:0'),
  tensor([[[1.]]], device='cuda:0'),
  tensor([[[1.]]], device='cuda:0'),
  tensor([[[1.]]], device='cuda:0'),
  tensor([[[1.]]], device='cuda:0'),
  tensor([[[1.]]], device='cuda:0'),
  tensor([[[1.]]], device='cuda:0'),
  tensor([[[1.]]], device='cuda:0')])

ИИИИ давайте также научимся считать самую популярную метрику для перевода -- BLEU (https://en.wikipedia.org/wiki/BLEU)

В общем-то, вам повезло -- ее писать руками скучно, да и nltk ее написало за вас:

In [0]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

def compute_bleu(inp_lines, out_lines):
    """ Estimates corpora-level BLEU score of model's translations given inp and reference out """
    translations = [[translate(line)[0]] for line in inp_lines]
    return corpus_bleu(translations, out_lines) * 100

In [0]:
a = [[SRC.vocab.itos[j] for j in next(iter(test_iterator)).src[:, i]] for i in range(64)]
b = [[TRG.vocab.itos[j] for j in next(iter(test_iterator)).trg[:, i]] for i in range(64)]

In [171]:
compute_bleu(a, b)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


7.46387198903277

In [0]:
corpus_bleu([[['man', 'works']], [['man', 'works']]], [['man', 'works', 'cdssdcsd', 'dcscsdc', 'shit'], ['man', 'works']]) * 100

Если вы реализовали несколько методов аттеншена, опишите каждый из них и полученную метрику на нем в отчете.

<Место для вашего отчета>

In [0]:
source = [next(iter(test_iterator)).src[:, 3]]
target = next(iter(test_iterator)).trg[:, 3]

In [0]:
[TRG.vocab.itos[j] for j in target]